# LangChain 表达式语言
- 一种在langchian之上封装的高级解释语言
- 简化链条开发，支撑真实生产环境而发明
- 更好的流式支持
- 轻松获取中间步骤
- 更好的异步支持
- 输入输出强验证
- 优化执行时间
- 支持重试和反馈
- 无缝追踪集成
- 无缝部署集成

# Runnable接口
- stream  流式返回响应接口
- invoke 给chain一个输入
- batch 给chain多个输入
- astream 以异步方式返回数据
- ainvoke 异步方式给chain输入
- abatch 异步方式给chain多个输入
- Astream_log 实时将中间步骤流式传输

In [1]:
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv()

True

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("给我讲一个关于 {topic}的笑话")
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "冰激凌"})

'为什么冰激凌会变得那么软？因为它总是在被批评，所以变得融化了！哈哈哈！'

In [4]:
prompt_value = prompt.invoke({"topic": "刺猬"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='给我讲一个关于 刺猬的笑话')])

In [5]:
prompt_value.to_messages()

[HumanMessage(content='给我讲一个关于 刺猬的笑话')]

In [6]:
prompt_value.to_string()

'Human: 给我讲一个关于 刺猬的笑话'

In [7]:
message = model.invoke(prompt_value)
message

AIMessage(content='为什么刺猬永远不会迟到？\n\n因为他总是能及时“刺”到目的地！😄😄😄', response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 22, 'total_tokens': 64, 'pre_token_count': 16384, 'pre_total': 124, 'adjust_total': 123, 'final_total': 1}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-427abfbf-9798-4f7a-9b67-935a569a3676-0')

# 使用llm的区别

In [9]:
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")
llm.invoke(prompt_value)

AIMessage(content='为什么刺猬不喜欢和别人分享食物呢？\n\n因为他们觉得吃了之后会“刺”倒别人！哈哈哈！', response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 22, 'total_tokens': 72, 'pre_token_count': 16384, 'pre_total': 124, 'adjust_total': 123, 'final_total': 1}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5740175b-39bd-41df-8f18-b5a302e3ce9e-0')

In [10]:
output_parser.invoke(message)

'为什么刺猬永远不会迟到？\n\n因为他总是能及时“刺”到目的地！😄😄😄'

# RAG Search Exampl
- 建立向量数据
- 使用RAG增强

In [11]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [12]:
chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

In [13]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [14]:
chain.invoke({"question": "where did harrison work", "language": "chinese"})

'Harrison 在 Kensho 工作。'